In [25]:
import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision import transforms, datasets
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from matplotlib import cm 
import numpy as np

from sklearn.datasets import load_iris

In [29]:
iris  = load_iris()

In [30]:
X = iris.data
Y = iris.target
X = torch.tensor(X, dtype = torch.float32)
Y = torch.tensor(Y, dtype = torch.int64)

In [40]:
dataset = TensorDataset(X,Y)
loader= DataLoader(dataset, batch_size=1, shuffle = True)

In [46]:
X.size(1)

4

In [6]:
# 오토인코더 모듈 정의
class Autoencoder(nn.Module):
    def __init__(self, X_train):
        super(Autoencoder, self).__init__()
        data_size = X_train.size(1)
        
        
        self.encoder = nn.Sequential( 
            nn.Linear(data_size, 32), #차원을 data_size에서 점차 줄여나갑니다.
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4),   
        )
        self.decoder = nn.Sequential(
            nn.Linear(4, 8), #디코더는 차원을 점차 data_size로 복원합니다.
            nn.ReLU(),
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, data_size),
            nn.Sigmoid(),       
        )

    def forward(self, x):
        encoded = self.encoder(x) 
        decoded = self.decoder(encoded) 
        return encoded, decoded

In [7]:
def Autoencoder_train(X_train, Y_train, X_test, epochs):
    
    X = torch.tensor(X_train, dtype = torch.float32)
    Y = torch.tensor(X_train, dtype = torch.int64)
    dataset = TensorDataset(X,Y)
    loader= DataLoader(dataset, batch_size=1, shuffle = True)
    
    #신경망 생성하기 
    autoencoder = Autoencoder()
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr = 0.005)
    criterion = nn.MSELoss()
    
    #train data로 학습
    autoencoder.train()
    for epoch in range(epochs):
        autoencoder.train()
        for step , (x, label) in enumerate(loader):
            y = x
            
            encoded, decoded = autoencoder(x)
            loss = criterion(decoded , y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    
    Y_test_pred = autoencoder(X_test)
    
    return y_test_pred
        